[View in Colaboratory](https://colab.research.google.com/github/upadhyayprakash/EIP/blob/master/PRAKASH_BATCH_3_ASSIGNMENT4B.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 1
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

164306944/170498071 [===========================>..] - ETA: 1s

170500096/170498071 [==============================] - 43s 0us/step


In [0]:
newdata = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

newdata.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.2
l = 10

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# START: Added Pooling & Dropout by Prakash

Second_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(First_Conv2D)
DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash


# START: Added Pooling & Dropout by Prakash
#BatchNorm = BatchNormalization()(input)
#relu = Activation('relu')(BatchNorm)
    
#Second_Conv2D = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

#DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
#AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#Last_Block = add_denseblock(Third_Block,  num_filter, dropout_rate)

output = output_layer(Third_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_3 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit_generator(newdata.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/50
782/782 [==============================] - 402s 514ms/step - loss: 1.7565 - acc: 0.3768 - val_loss: 1.4888 - val_acc: 0.4971
Epoch 2/50
122/782 [===>..........................] - ETA: 5:09 - loss: 1.4959 - acc: 0.4699

782/782 [==============================] - 392s 501ms/step - loss: 1.3612 - acc: 0.5120 - val_loss: 1.3091 - val_acc: 0.5682
Epoch 3/50
213/782 [=======>......................] - ETA: 4:27 - loss: 1.2074 - acc: 0.5686

782/782 [==============================] - 392s 501ms/step - loss: 1.1586 - acc: 0.5871 - val_loss: 1.4959 - val_acc: 0.5800
Epoch 4/50
247/782 [========>.....................] - ETA: 4:10 - loss: 1.0573 - acc: 0.6304

782/782 [==============================] - 392s 501ms/step - loss: 1.0279 - acc: 0.6374 - val_loss: 2.1721 - val_acc: 0.5399
Epoch 5/50
260/782 [========>.....................] - ETA: 4:04 - loss: 0.9572 - acc: 0.6641

782/782 [==============================] - 392s 501ms/step - loss: 0.9316 - acc: 0.6742 - val_loss: 1.7038 - val_acc: 0.5933
Epoch 6/50
265/782 [=========>....................] - ETA: 4:02 - loss: 0.8617 - acc: 0.6994

...

782/782 [==============================] - 396s 506ms/step - loss: 0.3559 - acc: 0.8766 - val_loss: 0.6726 - val_acc: 0.8353
Epoch 34/50
266/782 [=========>....................] - ETA: 4:04 - loss: 0.3539 - acc: 0.8781

782/782 [==============================] - 395s 505ms/step - loss: 0.3521 - acc: 0.8781 - val_loss: 0.5289 - val_acc: 0.8472
Epoch 35/50
266/782 [=========>....................] - ETA: 4:04 - loss: 0.3493 - acc: 0.8793

782/782 [==============================] - 396s 506ms/step - loss: 0.3471 - acc: 0.8791 - val_loss: 0.4531 - val_acc: 0.8763
Epoch 36/50
266/782 [=========>....................] - ETA: 4:05 - loss: 0.3373 - acc: 0.8811

782/782 [==============================] - 396s 506ms/step - loss: 0.3425 - acc: 0.8799 - val_loss: 0.9079 - val_acc: 0.7965
Epoch 37/50
266/782 [=========>....................] - ETA: 4:04 - loss: 0.3349 - acc: 0.8843

301/782 [==========>...................] - ETA: 3:48 - loss: 0.3393 - acc: 0.8844

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_Model_Prakash_Attempt_3.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('DNST_Model_Prakash_Attempt_3.h5')